# Säutsud pythonis

## Sissejuhatus

Käesoleva kirjatüki eesmärk on mudeldada pythonis säutsude teemasid. Selleks tehakse järgnevat:
 - laetakse twitterist säutsud alla
 - säutsud puhastatakse
 - ehitatakse teemadem udelid kasutades _Latent Dirichlet allocation_ algoritmi
 - tulemused visualiseeritakse

Esmalt defineerime hunniku funktsioone, mis teevad elu lihtsamaks.

In [1]:
def teksti_eraldaja(tweedid):
    """eraldab säutsude listist teksti. 
    sisend: säutsude metaandmete list, mis on tulnud funktsioonist allalaadija
    väljund: säutsude tekstide list"""
    tekst=[]
    for i in range(len(tweedid)):
        tekst.append(tweedid[i]['text'])  
    return tekst   

import math as math
def allalaadija(märksõna, kogus, keel='en'):
    """laeb twitterist säutsud alla
    sisend: -märksõna: otsisõna, mille kohta otsime twitterist säutsusid
    -kogus: mitu säutsu tahame. Kogus on 100 säutsu täpsusega (nt kogus=101, korjab 200 säutsu, kogus=99 korja 100 säutsu
    -keel: mis keelseid säutsusid tahame)
    väljund: - list tweetide andmetega"""
    loopideArv=math.ceil(kogus/100)
    print("Tsükkel nr:", str(1)+ ", veel tsükleid:",str(loopideArv-1))
    tweedid=twitter.search(q=märksõna,lang=keel, count=100)
    tulem=tweedid['statuses']
    minid=min_id(tweedid)
    for i in range(loopideArv-1):
        print("Tsükkel nr:", str(i+2)+ ", veel tsükleid:",str(loopideArv-i-2))
        uuedtweedid=twitter.search(q=märksõna,lang=keel, count=100, 
                                   max_id=minid)
        tulem+=uuedtweedid['statuses']
        minid=min_id(uuedtweedid)
    return tulem

def min_id(tweedid):
    """abifunktsioon funktsioonile allalaadija, mis leiab allalaaditud tweetide id miinimumi, selle saab
    anda sisendiks järgmisele loobile, et võtta tweedid, mille id on väiksem kui siin leitud id
    sisend: - tweetide sõnastik
    väljund: - tweetide sõnastiku id miinimum"""
    ids=[]
    for i in range(len(tweedid['statuses'])):
        ids.append(tweedid['statuses'][i]['id'])
    return min(ids)

from nltk.tokenize import RegexpTokenizer
import re
def tokeniseerija(tekstlist):
    """funktsioon, mis võtab sisendiks teksti listi ning teeb neist tokenid (iga sõna eraldi listi elemendiks) ning 
    eemaldab numbrid, kirjavahemärgid, lingid ja twitteri kasutajanimed ning teeb teksti väiketähtedeks
    sisend: - tekstilist (iga lause on 1 string)
    väljund: - tokenite list"""
    tokenizer = RegexpTokenizer(r'\w+')
    tokens=[]
    for i in range(len(tekstlist)):
        #eemaldame kasutajanimed ja lingid, numbrid ja kirjavahemärgid
        abimuutuja=re.sub(r'(^https?:\/\/.*[\r\n]*)|(@[A-Za-z0-9]+)|([0-9])|(\w+:\/\/\S+)', '', tekstlist[i].lower(), 
                          flags=re.MULTILINE)
        tokens.append(tokenizer.tokenize(abimuutuja))
    return tokens

from stop_words import get_stop_words
def puhastaja(tokenslist, myralist):
    """funktsioon, mis puhastab tokenite listi stoppsõnadest ja kasutaja poolt määratud sõnadest ning sõnadest, mille
    pikkus on alla 3
    sisend: - tokenslist: tokenite list
    - myralist: list sõnadest, mida tahetakse veel eemaldada
    väljund: - tokenite list, kus eelnev on eemaldatud"""
    stopsonad = get_stop_words('en')#inglise keel stoppsõnad
    stopsonad.extend(myralist)#paneme veel juurde
    #eemaldame stoppsõnad ja sõnad alla 2 täheärgi
    tokens_stopped=[]
    for j in range(len(tokenslist)):
        tokens_stopped.append([i for i in tokenslist[j] if not i in stopsonad and len(i)>2])   
    return tokens_stopped

from nltk.stem.porter import PorterStemmer
def stemming(tokens):
    """funktsioon, mis normaliseerib sõnad lõigates maha käände- ja pöördelõpud.
    sisend: - tokens: tokenite list
    väljudn: - normaliseeritud sõnade list"""
    p_stemmer = PorterStemmer()
    stemmed=[]
    for j in range(len(tokens)):
        stemmed.append([p_stemmer.stem(i) for i in tokens[j]]) 
    return stemmed

## Andmete korjamine 

Andmete korjamiseks kasutan eelloodud funktsioone.

In [2]:
from twython import Twython
#ühendus twitteriga, keyde ja salasõnade jaoks rega enda app: https://dev.twitter.com/ 
#ja loe twythoni kohta http://twython.readthedocs.io/en/latest/#
twitter = Twython('sinu_api_vaartus','sinu_api_vaartus','sinu_api_vaartus', 'sinu_api_vaartus')

In [ ]:
estonia=allalaadija('estonia', 2000)
latvia=allalaadija('latvia', 2000)
#salvestame, et igal korral ei peaks uue päringu tegema
import json
json.dump(estonia, open("estonia.txt",'w'))
json.dump(latvia, open("latvia.txt",'w'))

In [3]:
#laeme salvestatud failid üles
estonia = json.load(open("estonia.txt"))
latvia=json.load(open("latvia.txt"))

## Andmete töötlus ja puhastamine 

In [4]:
#eraldame teksti
estoniatkst=teksti_eraldaja(estonia)
latviatkst=teksti_eraldaja(latvia)

In [5]:
#tokeniseerime
esttoken=tokeniseerija(estoniatkst)
lattoken=tokeniseerija(latviatkst)

In [6]:
#puhastame
estpuhas=puhastaja(esttoken,['estonia'])
latpuhas=puhastaja(lattoken,['latvia'])

In [7]:
#stemming
eststem=stemming(estpuhas)
latstem=stemming(latpuhas)

## Mudeli ehitamine

Mudeli ehitamiseks katsetame erinevaid python pakette. Esmalt gensim pakett.

In [8]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
import warnings
warnings.filterwarnings('ignore')#et igasugu hoiatusi siin ei kuvaks

C:\Users\Risto\Anaconda3\lib\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Risto\Anaconda3\lib\site-packages\gensim\utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [9]:
estdictionary = Dictionary(eststem)
estcorpus = [estdictionary.doc2bow(text) for text in eststem]
#mudel
estmudel = LdaModel(corpus=estcorpus, id2word=estdictionary, 
                    iterations=500, num_topics=3,  
                    random_state=1)

In [10]:
#topicud sõnadega
estmudel.print_topics(num_topics=3, num_words=3)

[(0, '0.014*"farm" + 0.013*"end" + 0.013*"fur"'),
 (1, '0.015*"javier" + 0.015*"fernandez" + 0.013*"amp"'),
 (2, '0.017*"nato" + 0.017*"amp" + 0.015*"poland"')]

In [11]:
#interaktiivne visualiseerimine
import pyLDAvis.gensim
from gensim.models.coherencemodel import CoherenceModel
estldavis=pyLDAvis.gensim.prepare(estmudel, estcorpus, estdictionary)

C:\Users\Risto\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Users\Risto\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Users\Risto\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Users\Risto\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Users\Risto\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Users\Risto\Anaconda3\lib\site-packages\funcy\decora

In [13]:
pyLDAvis.display(estldavis)

In [14]:
#sama asi Läti tweetide puhul
latdictionary = Dictionary(latstem)
latcorpus = [latdictionary.doc2bow(text) for text in latstem]
#mudel
latmudel = LdaModel(corpus=latcorpus, id2word=latdictionary, 
                    iterations=500, num_topics=3, random_state=1)
#topic term
latmudel.get_topic_terms(2)
#topicud
latmudel.show_topics(3)
latmudel.print_topics(num_topics=3, num_words=3)
latldavis=pyLDAvis.gensim.prepare(latmudel, latcorpus, latdictionary) 
pyLDAvis.display(latldavis) 